In [ ]:
folder = 'multi_cased_L-12_H-768_A-12'
download_url = 'https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip'  # ссылка на скачивание модели

print('Downloading model...')
zip_path = '{}.zip'.format(folder)
!test -d $folder || (wget $download_url && unzip $zip_path)

!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

!pip install keras-bert

import sys
import numpy as np
from keras_bert import load_trained_model_from_checkpoint
import tokenization

config_path = folder+'/bert_config.json'
checkpoint_path = folder+'/bert_model.ckpt'
vocab_path = folder+'/vocab.txt'

import tensorflow as tf
tf.gfile = tf.io.gfile

tokenizer = tokenization.FullTokenizer(vocab_file=vocab_path, do_lower_case=False)

# загружаем модель
print('Loading model...')
model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)

print('OK')


--2021-05-22 17:19:12--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 2607:f8b0:4023:c06::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M  82.4MB/s    in 7.2s    

2021-05-22 17:19:20 (88.2 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflatin

In [ ]:
# входная фраза с закрытыми словами с помощью [MASK]
sentence = 'Я-то думал он принимает мою позицию, а он даже [MASK] не принимает'

print(sentence)


sentence = sentence.replace(' [MASK] ','[MASK]'); sentence = sentence.replace('[MASK] ','[MASK]'); sentence = sentence.replace(' [MASK]','[MASK]')
sentence = sentence.split('[MASK]')
tokens = ['[CLS]']

for i in range(len(sentence)):
    if i == 0:
        tokens = tokens + tokenizer.tokenize(sentence[i]) 
    else:
        tokens = tokens + ['[MASK]'] + tokenizer.tokenize(sentence[i]) 
tokens = tokens + ['[SEP]']

token_input = tokenizer.convert_tokens_to_ids(tokens)        

token_input = token_input + [0] * (512 - len(token_input))

mask_input = [0]*512
for i in range(len(mask_input)):
    if token_input[i] == 103:
        mask_input[i] = 1

seg_input = [0]*512


token_input = np.asarray([token_input])
mask_input = np.asarray([mask_input])
seg_input = np.asarray([seg_input])


predicts = model.predict([token_input, seg_input, mask_input])[0]
predicts = np.argmax(predicts, axis=-1)


predicts = predicts[0][:len(tokens)]
out = []

for i in range(len(mask_input[0])):
    if mask_input[0][i] == 1:
        out.append(predicts[i]) 

out = tokenizer.convert_ids_to_tokens(out)
out = ' '.join(out)               
out = tokenization.printable_text(out)
out = out.replace(' ##','')
print('Result:', out)

Я-то думал он принимает мою позицию, а он даже [MASK] не принимает
Result: себя
